# Setup - 
Run jupyter-notebook in env and pick link


In [2]:
import logging
# show both debug and warning messages
logging.basicConfig(level=logging.DEBUG)

In [3]:
import sys
sys.path.insert(0, "/NS/llm-pretraining/work/afkhan/tokensmith")

In [4]:
import sys
sys.path.insert(0, "/NS/llm-pretraining/work/afkhan/USC_Colab/gpt-neox")

In [5]:
from transformers import AutoTokenizer
TOKENIZER_NAME_OR_PATH = "EleutherAI/gpt-neox-20b"
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME_OR_PATH, add_eos_token=True)

/NS/venvs/work/afkhan/neox_updated_env/lib/python3.11/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /EleutherAI/gpt-neox-20b/resolve/main/tokenizer_config.json HTTP/1.1" 307 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /api/resolve-cache/models/EleutherAI/gpt-neox-20b/c292233c833e336628618a88a648727eb3dff0a7/tokenizer_config.json HTTP/1.1" 200 0
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
# import os
# import torch
# import torch.distributed as dist

# def setup():
#     # Set required env vars manually
#     os.environ['RANK'] = '0'
#     os.environ['WORLD_SIZE'] = '1'
#     os.environ['MASTER_ADDR'] = '127.0.0.1'
#     os.environ['MASTER_PORT'] = '29500'

#     dist.init_process_group(backend='nccl', init_method='env://')

# setup()

# import torch
# import os

# os.environ['MASTER_ADDR'] = '127.0.0.1'
# os.environ['MASTER_PORT'] = '29500'
# torch.distributed.init_process_group(backend='nccl', rank=0, world_size=1)

In [7]:
from tokensmith.manager import DatasetManager

[2025-07-01 09:33:05,000] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/NS/venvs/work/afkhan/neox_updated_env/lib/python3.11/site-packages/deepspeed/runtime/zero/linear.py:47: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @autocast_custom_fwd
/NS/venvs/work/afkhan/neox_updated_env/lib/python3.11/site-packages/deepspeed/runtime/zero/linear.py:66: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @autocast_custom_bwd
DEBUG:git.cmd:Popen(['git', 'version'], cwd=/NS/llm-pretraining/work/afkhan/tokensmith/artifacts, stdin=None, shell=False, universal_newlines=False)
DEBUG:git.cmd:Popen(['git', 'version'], cwd=/NS/llm-pretraining/work/afkhan/tokensmith/artifacts, stdin=None, shell=False, universal_newlines=False)
DEBUG:wandb.docker.auth:Trying paths: ['/home/afkhan/.docker/config.json', '/home/afkhan/.dockercfg']
DEBUG:wandb.docker.auth:Found file at path: /home/afkhan/.docker/c

In [8]:
dataset_manager = DatasetManager()

# Setup Search

In [9]:
# dataset_manager.setup_search(
#     bin_file_path="/NS/llm-pretraining/work/afkhan/tokensmith/artifacts/data_tokenized_text_document.bin",
#     search_index_save_path="/NS/llm-pretraining/work/afkhan/tokensmith/artifacts/search_index_text_document.idx",
#     vocab=2**32,
#     verbose=True,
#     reuse=True,
# )

In [10]:
dataset_manager.setup_edit_inspect_sample_export(
    dataset_prefix='/NS/llm-pretraining/work/afkhan/tokensmith/artifacts/data_tokenized_text_document',
    batch_info_save_prefix='/NS/llm-pretraining/work/afkhan/tokensmith/artifacts/batch_info',
    train_iters=100,
    train_batch_size=16,
    train_seq_len=2048,
    seed=42,
    splits_string='990,5,5',
    packing_impl='packed',
    allow_chopped=True,
)

DEBUG:tokensmith.utils:Initializing WriteableMMapIndexedDataset with pointer: /NS/llm-pretraining/work/afkhan/tokensmith/artifacts/data_tokenized_text_document.bin and index: /NS/llm-pretraining/work/afkhan/tokensmith/artifacts/data_tokenized_text_document.idx


    warming up index mmap file...
    reading sizes...
    reading pointers...
    reading document index...


DEBUG:tokensmith.utils:Loading doc/sample/shuffle indexes with prefix: /NS/llm-pretraining/work/afkhan/tokensmith/artifacts/batch_info_train_indexmap_1600ns_2048sl_42s_packedpi_ac*


# Inspect

In [20]:
resp = dataset_manager.inspect.inspect_sample_by_id(sample_id=10, return_doc_details=True, return_detokenized = True, tokenizer=tokenizer)
print(resp)

(' day, they find a big club on the grass. It is brown and heavy. "Look, a club!" Lily says. "Let\'s play with it!" "OK!" Ben says. "What can we do with it?" Lily thinks for a moment. Then she has an idea. "We can spin it!" she says. "Like this!" She holds the club with both hands and turns around fast. The club makes a whoosh sound in the air. "Wow, that looks fun!" Ben says. "Can I try?" "Sure!" Lily says. She gives the club to Ben. Ben tries to spin it, but it is too heavy for him. He falls down and drops the club. "Ouch!" Ben says. "That hurts!" Lily laughs. She helps Ben get up. "You are lazy!" she says. "You need to be strong to spin the club!" Ben makes a face. "I am not lazy!" he says. "I am just small. You are big and strong. That\'s not fair!" Lily smiles. She hugs Ben. "Don\'t be sad," she says. "We can play something else. How about we swing on the swings?" "OK!" Ben says. "That sounds fun!" They leave the club on the grass and run to the swings. They swing high and low, an

In [24]:
resp = dataset_manager.inspect.inspect_sample_by_batch(batch_id=1, batch_size=16, return_doc_details=False, return_detokenized=True, tokenizer=tokenizer)

for i, r in enumerate(resp):
    print(f"Sequence {i}: {r}")

Sequence 0:  you too," she says. "But you have to promise me that you won't do that again. And you have to help me clean up the mess. And you have to wait for dinner. No more cookies for you today." Lily and Ben agree. They promise to be good. They help Mom clean up the kitchen. They wait for dinner. They don't touch the cookies anymore. They learn their lesson. They also learn that cookies are brown and sweet and crunchy. And that they taste better when they are shared.<|endoftext|>Once upon a time, there was a little girl named Lily who loved to collect shiny things. One day, she found a pretty crystal on the ground and put it in her pocket. She was so happy and showed it to everyone she met. But one day, Lily forgot where she put the crystal. She looked everywhere but couldn't find it. She was troubled and sad. Her friend, Timmy, asked her what was wrong. She told him about the lost crystal. "Don't worry, Lily," Timmy said. "Maybe someone else found it and will take good care of it.

# Sample

In [30]:
indices = [10, 50, 99, 133]

resp = dataset_manager.sample.get_samples_by_indices(indices, return_doc_details=False, return_detokenized=True, tokenizer=tokenizer)

for i, r in enumerate(resp):
    print(f"Sample at index {indices[i]}: {r}")

Sample at index 10:  day, they find a big club on the grass. It is brown and heavy. "Look, a club!" Lily says. "Let's play with it!" "OK!" Ben says. "What can we do with it?" Lily thinks for a moment. Then she has an idea. "We can spin it!" she says. "Like this!" She holds the club with both hands and turns around fast. The club makes a whoosh sound in the air. "Wow, that looks fun!" Ben says. "Can I try?" "Sure!" Lily says. She gives the club to Ben. Ben tries to spin it, but it is too heavy for him. He falls down and drops the club. "Ouch!" Ben says. "That hurts!" Lily laughs. She helps Ben get up. "You are lazy!" she says. "You need to be strong to spin the club!" Ben makes a face. "I am not lazy!" he says. "I am just small. You are big and strong. That's not fair!" Lily smiles. She hugs Ben. "Don't be sad," she says. "We can play something else. How about we swing on the swings?" "OK!" Ben says. "That sounds fun!" They leave the club on the grass and run to the swings. They swing h

In [31]:
batch_ids = [1, 7, 89, 100]

resp = dataset_manager.sample.get_batches_by_ids(batch_ids, batch_size=2, return_doc_details=False, return_detokenized=True, tokenizer=tokenizer)

for i, r in enumerate(resp):
    print(f"Batch ID {batch_ids[i]}")
    for j, seq in enumerate(r):
        print(f"  Sequence {j}: {seq}")


Batch ID 1
  Sequence 0:  agreed to marry him. They had a wonderful wedding and were very happy together. They lived happily ever after in the chamber with the thick walls. The End.<|endoftext|>Once upon a time, there was a little girl named Lily. One day, she went to the store with her mommy to buy a candle for their home. They picked out a pretty red candle with a rough texture. As they were leaving the store, a man stopped them and asked for some money. Lily's mommy said, "I'm sorry, I can't pay you right now." The man looked sad and walked away. Later that night, Lily and her mommy lit the candle and it made their home smell nice. But then, the lights went out! It was very dark and scary. Lily's mommy remembered the man from the store and said, "I wish we could have helped him. Maybe he could have fixed our lights." Suddenly, there was a knock on the door. It was the man from the store! He had fixed their lights and said, "Thank you for helping me pay for my dinner. I saw you looki

In [10]:
import numpy as np

RNG = np.random.default_rng(42)

In [12]:
dataset_manager.edit.inject_and_preview(
    text = 'This is a test sentence.',
    tokenizer=tokenizer,
    injection_loc=699,
    injection_type="seq_shuffle",
    rng=RNG,
    add_eos_token=True,
    dry_run=False,
)

Dummy sample: [1552  310  247 1071 6197   15    0]
Training sample 699
Sample consists of segments from 12 documents
Raw sample: [16522   497   594 ...  5972    15   380]
---
Decoded sample:  Dad were so excited to show me their new thing. It was tall and wide with a shapely top. It was bright red and had pretty decorations. I was so amazed! Mum and Dad had designed a beautiful red chair for me. I squealed with delight as I hugged them and spun around in it. It was the best thing ever!<|endoftext|>Once upon a time, there was a little bird with gray feathers. The bird loved to fly high in the sky and sit on a branch. One day, the bird saw a big, juicy worm on the ground. The bird wanted to eat the worm but his mother told him it was not safe to eat worms on the ground. She said only worms on the branch were safe to eat. But the little bird didn't listen to his mother and flew down to the ground to eat the worm. As he was eating, a big, mean cat came and chased after him. The little bird

DEBUG:tokensmith.utils:>>> Inserting 72 of 2049 tokens starting at position 8562636
DEBUG:tokensmith.utils:>>> Injection Data Shape (72,) | Injection Data Dtype uint16
DEBUG:tokensmith.utils:>>> Inserting 189 of 2049 tokens starting at position 7462992
DEBUG:tokensmith.utils:>>> Inserting 152 of 2049 tokens starting at position 4710212
DEBUG:tokensmith.utils:>>> Inserting 242 of 2049 tokens starting at position 3269832
DEBUG:tokensmith.utils:>>> Inserting 173 of 2049 tokens starting at position 1322428
DEBUG:tokensmith.utils:>>> Inserting 188 of 2049 tokens starting at position 4877468
DEBUG:tokensmith.utils:>>> Inserting 143 of 2049 tokens starting at position 3814036
DEBUG:tokensmith.utils:>>> Inserting 227 of 2049 tokens starting at position 1944972
DEBUG:tokensmith.utils:>>> Inserting 205 of 2049 tokens starting at position 8853872
DEBUG:tokensmith.utils:>>> Inserting 211 of 2049 tokens starting at position 1577198
DEBUG:tokensmith.utils:>>> Inserting 156 of 2049 tokens starting at

Training sample 699 after injection
Raw sample: [ 921  479  616 ...  285 8513 5972]
---
Decoded sample:  show me their new thing. It was tall and wide with a shapely top. It was bright red and had pretty decorations. I was so amazed! Mum and Dad had designed a beautiful red chair for me. I squealed with delight as I hugged them and spun around in it. It was the best thing ever!<|endoftext|>This is a test sentence.<|endoftext|>Once upon a time, there was a little bird with gray feathers. The bird loved to fly high in the sky and sit on a branch. One day, the bird saw a big, juicy worm on the ground. The bird wanted to eat the worm but his mother told him it was not safe to eat worms on the ground. She said only worms on the branch were safe to eat. But the little bird didn't listen to his mother and flew down to the ground to eat the worm. As he was eating, a big, mean cat came and chased after him. The little bird tried to fly away, but he was too slow. The cat caught him and ate him f

In [ ]:
tokenizer.decode([1552 , 310 , 247 ,1071 ,6197 ,  15])

'This is a test sentence.'

In [ ]:
tokenizer('This is a test sequence.', add_special_tokens=True)

{'input_ids': [1552, 310, 247, 1071, 3425, 15], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer.eos_token_id

0

In [ ]:
test_sample = list(tokenizer("big")["input_ids"])

In [ ]:
test_sample

[2760]

In [ ]:
dataset_manager.search.count(test_sample)

AttributeError: 'NoneType' object has no attribute 'count'

In [ ]:
from tokengrams import MemmapIndex

# Create a new index from an on-disk corpus of u16 tokens and save it to a .idx file. 
# Set verbose to true to include a progress bar for the index sort.
index = MemmapIndex.build(
    "/NS/llm-pretraining/work/afkhan/tokensmith/artifacts/data_tokenized_text_document.bin",
    "/NS/llm-pretraining/work/afkhan/tokensmith/artifacts/data_tokenized_text_document_search_2_32.idx",
    vocab=2**16,
)

# True for any valid index.
print(index.is_sorted())
  

Sorting indices...
True


In [ ]:
# Get the count of "hello world" in the corpus.
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")
print(index.count(tokenizer.encode("Once")))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


15326


In [ ]:
tokenizer.encode("once")

[19131]

In [ ]:
tokenizer.encode("big")

[2760]

# Policy-based Sampling Examples

Examples of using `get_samples_by_policy` and `get_batches_by_policy` with custom sampling policies.

In [34]:
import numpy as np
import random

In [35]:
# Define some example policy functions for sampling

def random_sample_policy(num_samples, max_index, rng_seed=42):
    """
    Policy function that returns random sample indices.
    
    Args:
        num_samples: Number of samples to return
        max_index: Maximum index value (exclusive)
        rng_seed: Random seed for reproducibility
    
    Returns:
        List of random sample indices
    """
    rng = np.random.default_rng(rng_seed)
    return rng.integers(0, max_index, size=num_samples).tolist()

def sequential_sample_policy(start_index, num_samples):
    """
    Policy function that returns sequential sample indices.
    
    Args:
        start_index: Starting index
        num_samples: Number of consecutive samples to return
    
    Returns:
        List of sequential sample indices
    """
    return list(range(start_index, start_index + num_samples))

def sparse_sample_policy(start_index, num_samples, step_size=10):
    """
    Policy function that returns sparsely distributed sample indices.
    
    Args:
        start_index: Starting index
        num_samples: Number of samples to return
        step_size: Step size between samples
    
    Returns:
        List of sparse sample indices
    """
    return [start_index + i * step_size for i in range(num_samples)]

In [36]:
# Example 1: get_samples_by_policy with random sampling
print("=== Random Sample Policy Example ===")

# Get 5 random samples using the random policy
random_samples = dataset_manager.sample.get_samples_by_policy(
    policy_fn=random_sample_policy,
    num_samples=5,
    max_index=1000,  # Assuming we have at least 1000 samples
    rng_seed=42,
    return_detokenized=True,
    tokenizer=tokenizer
)

for i, sample in enumerate(random_samples):
    print(f"Random Sample {i+1}: {sample[:100]}...")  # Show first 100 chars

=== Random Sample Policy Example ===
Random Sample 1:  with a frown on its face. She was very curious and wanted to know why the tree was sad. Lily asked ...
Random Sample 2:  on the slide. She didn't want to hurt it, so she gently picked it up. But then a boy came and said,...
Random Sample 3: uffy's ball. He bit it hard and the ball popped. Fluffy was sad and ran away. Spot felt bad. He want...
Random Sample 4: . Timmy did not like going to the doctor because he was scared of shots. He hated shots. When Timmy ...
Random Sample 5:  said, "Can I go play in it?" Her mom said, "Of course! But, put on your warm clothes first." The li...


NameError: name 'dataset_manager' is not defined

In [37]:
# Example 2: get_samples_by_policy with sequential sampling
print("\n=== Sequential Sample Policy Example ===")

# Get 3 consecutive samples starting from index 100
sequential_samples = dataset_manager.sample.get_samples_by_policy(
    policy_fn=sequential_sample_policy,
    start_index=100,
    num_samples=3,
    return_detokenized=True,
    tokenizer=tokenizer
)

for i, sample in enumerate(sequential_samples):
    print(f"Sequential Sample {i+1} (index {100+i}): {sample[:100]}...")  # Show first 100 chars


=== Sequential Sample Policy Example ===
Sequential Sample 1 (index 100):  and peeked over the grass to get a better look. Sally asked the sheep, “What’s your name?” The youn...
Sequential Sample 2 (index 101):  it." Lily nodded and put the pebble in her pocket. She played for a while and then went back inside...
Sequential Sample 3 (index 102):  better than the brick! Mandy was very pleased with her new snack. She ate the carrot and thanked th...


In [38]:
# Example 3: get_samples_by_policy with sparse sampling and document details
print("\n=== Sparse Sample Policy with Document Details Example ===")

# Get sparse samples with document details
sparse_samples = dataset_manager.sample.get_samples_by_policy(
    policy_fn=sparse_sample_policy,
    start_index=50,
    num_samples=3,
    step_size=25,  # Every 25th sample
    return_doc_details=True,
    return_detokenized=True,
    tokenizer=tokenizer
)

for i, (sample, doc_details) in enumerate(sparse_samples):
    sample_index = 50 + i * 25
    print(f"Sparse Sample {i+1} (index {sample_index}):")
    print(f"  Document details: {doc_details}")
    print(f"  Text: {sample[:100]}...")  # Show first 100 chars
    print()


=== Sparse Sample Policy with Document Details Example ===
Sparse Sample 1 (index 50):
  Document details: {'doc_index_f': 14417, 'doc_index_l': 14425, 'offset_f': 215, 'offset_l': 176}
  Text:  Ben's car fell on the ground and broke. The wheel came off and the paint scratched. "Uh oh!" Lily s...

Sparse Sample 2 (index 75):
  Document details: {'doc_index_f': 16055, 'doc_index_l': 16064, 'offset_f': 215, 'offset_l': 36}
  Text:  the museum. Sam was very happy that she was able to help BRO when he was broken. The end.<|endoftex...

Sparse Sample 3 (index 100):
  Document details: {'doc_index_f': 21659, 'doc_index_l': 21670, 'offset_f': 45, 'offset_l': 103}
  Text:  and peeked over the grass to get a better look. Sally asked the sheep, “What’s your name?” The youn...



In [39]:
# Define policy functions for batch sampling

def random_batch_policy(num_batches, max_batch_id, rng_seed=42):
    """
    Policy function that returns random batch IDs.
    
    Args:
        num_batches: Number of batches to return
        max_batch_id: Maximum batch ID value (exclusive)
        rng_seed: Random seed for reproducibility
    
    Returns:
        List of random batch IDs
    """
    rng = np.random.default_rng(rng_seed)
    return rng.integers(0, max_batch_id, size=num_batches).tolist()

def sequential_batch_policy(start_batch_id, num_batches):
    """
    Policy function that returns sequential batch IDs.
    
    Args:
        start_batch_id: Starting batch ID
        num_batches: Number of consecutive batches to return
    
    Returns:
        List of sequential batch IDs
    """
    return list(range(start_batch_id, start_batch_id + num_batches))

def stride_batch_policy(start_batch_id, num_batches, stride=5):
    """
    Policy function that returns batch IDs with a specific stride.
    
    Args:
        start_batch_id: Starting batch ID
        num_batches: Number of batches to return
        stride: Stride between batch IDs
    
    Returns:
        List of strided batch IDs
    """
    return [start_batch_id + i * stride for i in range(num_batches)]

In [40]:
# Example 4: get_batches_by_policy with random batch sampling
print("\n=== Random Batch Policy Example ===")

# Get 2 random batches with batch size 3
random_batches = dataset_manager.sample.get_batches_by_policy(
    policy_fn=random_batch_policy,
    batch_size=3,
    num_batches=2,
    max_batch_id=50,  # Assuming we have at least 50 batches
    rng_seed=123,
    return_detokenized=True,
    tokenizer=tokenizer
)

for batch_idx, batch in enumerate(random_batches):
    print(f"Random Batch {batch_idx + 1}:")
    for sample_idx, sample in enumerate(batch):
        print(f"  Sample {sample_idx + 1}: {sample[:80]}...")  # Show first 80 chars
    print()


=== Random Batch Policy Example ===
Random Batch 1:
  Sample 1:  thing happened. Lily's little brother came running and accidentally stepped on ...
  Sample 2: . She had gone to the office for a minute. Lily had an idea. "Let's steal some c...
  Sample 3:  agreed to marry him. They had a wonderful wedding and were very happy together....

Random Batch 2:
  Sample 1:  better than the brick! Mandy was very pleased with her new snack. She ate the c...
  Sample 2:  ran to the house. She was cold and sad. She wished she had listened to Tom. She...
  Sample 3:  you done? This was my special doll. How could you be so careless and selfish? Y...



In [41]:
# Example 5: get_batches_by_policy with sequential batch sampling
print("=== Sequential Batch Policy Example ===")

# Get 2 consecutive batches starting from batch ID 10
sequential_batches = dataset_manager.sample.get_batches_by_policy(
    policy_fn=sequential_batch_policy,
    batch_size=4,
    start_batch_id=10,
    num_batches=2,
    return_detokenized=True,
    tokenizer=tokenizer
)

for batch_idx, batch in enumerate(sequential_batches):
    batch_id = 10 + batch_idx
    print(f"Sequential Batch {batch_idx + 1} (Batch ID {batch_id}):")
    for sample_idx, sample in enumerate(batch):
        sample_id = batch_id * 4 + sample_idx  # Assuming batch_size=4
        print(f"  Sample {sample_idx + 1} (ID {sample_id}): {sample[:80]}...")  # Show first 80 chars
    print()

=== Sequential Batch Policy Example ===
Sequential Batch 1 (Batch ID 10):
  Sample 1 (ID 40): . The ball is empty and flat. "Oh no, we broke the ball!" Lily says. "I am sorry...
  Sample 2 (ID 41):  dad said they could go and play on the sand and look for shells. Anna and Ben w...
  Sample 3 (ID 42):  when she saw a group of people waving their arms in the air. She walked closer ...
  Sample 4 (ID 43):  the birds. Sometimes, at night, they saw the moon and the stars. They thought t...

Sequential Batch 2 (Batch ID 11):
  Sample 1 (ID 44):  "Love you, sis." She put a blanket on him and they went home.<|endoftext|>Once ...
  Sample 2 (ID 45):  Tom. He looked very sad and sorry. She decided to forgive him. She looked at th...
  Sample 3 (ID 46):  their mom. They tell her about the truck and the hose and the horn. They had fu...
  Sample 4 (ID 47):  feel better. So, he decided to take Teddy to the toy doctor to get his zipper f...



In [42]:
# Example 6: get_batches_by_policy with stride batch sampling and document details
print("=== Stride Batch Policy with Document Details Example ===")

# Get 2 batches with stride 3 and include document details
stride_batches = dataset_manager.sample.get_batches_by_policy(
    policy_fn=stride_batch_policy,
    batch_size=2,
    start_batch_id=5,
    num_batches=2,
    stride=3,  # Skip 2 batches between selected batches
    return_doc_details=True,
    return_detokenized=True,
    tokenizer=tokenizer
)

for batch_idx, batch in enumerate(stride_batches):
    batch_id = 5 + batch_idx * 3  # stride=3
    print(f"Stride Batch {batch_idx + 1} (Batch ID {batch_id}):")
    for sample_idx, (sample, doc_details) in enumerate(batch):
        sample_id = batch_id * 2 + sample_idx  # Assuming batch_size=2
        print(f"  Sample {sample_idx + 1} (ID {sample_id}):")
        print(f"    Document details: {doc_details}")
        print(f"    Text: {sample[:60]}...")  # Show first 60 chars
    print()

=== Stride Batch Policy with Document Details Example ===
Stride Batch 1 (Batch ID 5):
  Sample 1 (ID 10):
    Document details: {'doc_index_f': 7983, 'doc_index_l': 7994, 'offset_f': 16, 'offset_l': 4}
    Text:  day, they find a big club on the grass. It is brown and hea...
  Sample 2 (ID 11):
    Document details: {'doc_index_f': 13066, 'doc_index_l': 13077, 'offset_f': 97, 'offset_l': 30}
    Text:  and dad said it seemed too dangerous, so they said no. The ...

Stride Batch 2 (Batch ID 8):
  Sample 1 (ID 16):
    Document details: {'doc_index_f': 9361, 'doc_index_l': 9371, 'offset_f': 639, 'offset_l': 30}
    Text:  you too," she says. "But you have to promise me that you wo...
  Sample 2 (ID 17):
    Document details: {'doc_index_f': 13023, 'doc_index_l': 13035, 'offset_f': 245, 'offset_l': 4}
    Text:  and Dad's hands. They were alone in the crowd. "Mom! Dad! W...



In [43]:
# Example 7: Using lambda functions as policies
print("=== Lambda Policy Functions Example ===")

# Example with lambda for prime-numbered indices
prime_indices = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]

# Use lambda to create a simple policy
lambda_samples = dataset_manager.sample.get_samples_by_policy(
    policy_fn=lambda indices_list: indices_list[:5],  # Just take first 5 from the list
    indices_list=prime_indices,
    return_detokenized=True,
    tokenizer=tokenizer
)

print("Samples at prime indices (first 5):")
for i, sample in enumerate(lambda_samples):
    print(f"Prime Index {prime_indices[i]}: {sample[:70]}...")  # Show first 70 chars

print("\n" + "="*50)

# Lambda policy for batch IDs that are powers of 2
power_of_2_batches = dataset_manager.sample.get_batches_by_policy(
    policy_fn=lambda base, count: [base ** i for i in range(1, count + 1)],
    batch_size=2,
    base=2,
    count=3,  # 2^1, 2^2, 2^3 = batches 2, 4, 8
    return_detokenized=True,
    tokenizer=tokenizer
)

print("Batches at power-of-2 IDs:")
for batch_idx, batch in enumerate(power_of_2_batches):
    batch_id = 2 ** (batch_idx + 1)
    print(f"Batch ID {batch_id}:")
    for sample_idx, sample in enumerate(batch):
        print(f"  Sample {sample_idx + 1}: {sample[:60]}...")
    print()

=== Lambda Policy Functions Example ===
Samples at prime indices (first 5):
Prime Index 2:  agreed to marry him. They had a wonderful wedding and were very happy...
Prime Index 3:  sleep, Maggie's mommy saw something very rare and wet. It was raining...
Prime Index 5: Let's look at the pictures. They might tell us something." Lila and Be...
Prime Index 7:  said. Bella was shocked and excited. She hugged her parents and thank...
Prime Index 11:  and dad said it seemed too dangerous, so they said no. The boy didn't...

Batches at power-of-2 IDs:
Batch ID 2:
  Sample 1:  Benny learned that not all big, wild animals are scary. Som...
  Sample 2: Let's look at the pictures. They might tell us something." L...

Batch ID 4:
  Sample 1:  park. They see the slide and the swing. They wish they had ...
  Sample 2:  run around the faucet, letting the water spray all over him...

Batch ID 8:
  Sample 1:  you too," she says. "But you have to promise me that you wo...
  Sample 2:  and Dad's hands. Th

# Export Examples

Examples of using the export functionality to save datasets in different formats (JSONL and CSV) with various selection strategies.

In [10]:
from pathlib import Path
import os
import json

# Create export directory if it doesn't exist
export_dir = Path("/NS/llm-pretraining/work/afkhan/tokensmith/exports")
export_dir.mkdir(exist_ok=True)

print(f"Export directory: {export_dir}")
print(f"Export directory exists: {export_dir.exists()}")

Export directory: /NS/llm-pretraining/work/afkhan/tokensmith/exports
Export directory exists: True


In [11]:
# Example 1: Export specific sequences to JSONL
print("=== Export Specific Sequences Example ===")

# Define sequence indices to export
sequence_indices = [10, 25, 50, 75, 100]
output_path_jsonl = export_dir / "specific_sequences.jsonl"

# Export to JSONL format with detokenized text
dataset_manager.export.export_sequences(
    sequence_indices=sequence_indices,
    output_path=str(output_path_jsonl),
    format_type="jsonl",
    return_detokenized=True,
    tokenizer=tokenizer,
    include_doc_details=False
)

print(f"Exported {len(sequence_indices)} sequences to: {output_path_jsonl}")

# Read and display first few lines to verify
with open(output_path_jsonl, 'r') as f:
    for i, line in enumerate(f):
        if i < 2:  # Show first 2 records
            record = json.loads(line)
            print(f"Record {i+1}: Index {record['index']}, Content: {record['content'][:80]}...")
        else:
            break

=== Export Specific Sequences Example ===
Exported 5 sequences to: /NS/llm-pretraining/work/afkhan/tokensmith/exports/specific_sequences.jsonl
Record 1: Index 0, Content:  day, they find a big club on the grass. It is brown and heavy. "Look, a club!" ...
Record 2: Index 1, Content:  that the sun made droplets scatter off of their backs! They felt so refreshed i...


In [12]:
# Example 2: Export specific sequences to CSV with document details
print("\n=== Export Sequences with Document Details to CSV ===")

# Different sequence indices
sequence_indices_detailed = [5, 15, 30, 45]
output_path_csv = export_dir / "sequences_with_details.csv"

# Export to CSV format with document details
dataset_manager.export.export_sequences(
    sequence_indices=sequence_indices_detailed,
    output_path=str(output_path_csv),
    format_type="csv",
    return_detokenized=True,
    tokenizer=tokenizer,
    include_doc_details=True
)

print(f"Exported {len(sequence_indices_detailed)} sequences with document details to: {output_path_csv}")

# Read and display CSV content
import pandas as pd
df = pd.read_csv(output_path_csv)
print(f"CSV shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print(f"First row preview:")
print(f"  Index: {df.iloc[0]['index']}")
print(f"  Content (first 80 chars): {df.iloc[0]['content'][:80]}...")
if 'doc_details' in df.columns:
    print(f"  Doc details: {df.iloc[0]['doc_details']}")


=== Export Sequences with Document Details to CSV ===
Exported 4 sequences with document details to: /NS/llm-pretraining/work/afkhan/tokensmith/exports/sequences_with_details.csv
CSV shape: (4, 6)
Columns: ['index', 'content', 'doc_index_f', 'doc_index_l', 'offset_f', 'offset_l']
First row preview:
  Index: 0
  Content (first 80 chars): Let's look at the pictures. They might tell us something." Lila and Ben look at ...


In [13]:
# Example 3: Export specific batches
print("\n=== Export Specific Batches Example ===")

# Define batch IDs to export
batch_ids = [1, 5, 10]
batch_size = 4
output_path_batches = export_dir / "specific_batches.jsonl"

# Export specific batches to JSONL
dataset_manager.export.export_batches(
    batch_ids=batch_ids,
    batch_size=batch_size,
    output_path=str(output_path_batches),
    format_type="jsonl",
    return_detokenized=True,
    tokenizer=tokenizer,
    include_doc_details=False,
    flatten_batches=False  # Keep batch structure
)

print(f"Exported {len(batch_ids)} batches (batch_size={batch_size}) to: {output_path_batches}")

# Read and display structure
with open(output_path_batches, 'r') as f:
    lines = f.readlines()
    print(f"Total records in file: {len(lines)}")
    print(f"Expected records: {len(batch_ids) * batch_size} (3 batches × 4 samples each)")
    
    # Show first record from each batch
    for i in range(min(3, len(lines))):
        record = json.loads(lines[i])
        print(f"Batch record {i+1}: Index {record['index']}, Content: {record['content'][:60]}...")


=== Export Specific Batches Example ===


AttributeError: 'list' object has no attribute 'tolist'

In [ ]:
# Example 4: Export batches with flattening
print("\n=== Export Flattened Batches Example ===")

# Export the same batches but flattened into a single list
batch_ids_flat = [2, 6]
output_path_flat = export_dir / "flattened_batches.jsonl"

dataset_manager.export.export_batches(
    batch_ids=batch_ids_flat,
    batch_size=3,
    output_path=str(output_path_flat),
    format_type="jsonl",
    return_detokenized=True,
    tokenizer=tokenizer,
    include_doc_details=False,
    flatten_batches=True  # Flatten all samples into single list
)

print(f"Exported {len(batch_ids_flat)} batches (flattened) to: {output_path_flat}")

# Verify flattened structure
with open(output_path_flat, 'r') as f:
    lines = f.readlines()
    print(f"Total flattened records: {len(lines)}")
    
    # Show a few records
    for i in range(min(3, len(lines))):
        record = json.loads(lines[i])
        print(f"Flattened record {i+1}: Index {record['index']}, Content: {record['content'][:60]}...")


=== Export Flattened Batches Example ===
Exported 2 batches (flattened) to: /NS/llm-pretraining/work/afkhan/tokensmith/exports/flattened_batches.jsonl
Total flattened records: 6
Flattened record 1: Index 0, Content:  baby, saving them from certain injury. The creative baby fe...
Flattened record 2: Index 1, Content:  said. Bella was shocked and excited. She hugged her parents...
Flattened record 3: Index 2, Content:  park. They see the slide and the swing. They wish they had ...


In [ ]:
# Example 5: Export sequence range
print("\n=== Export Sequence Range Example ===")

# Export a range of consecutive sequences
start_idx = 100
end_idx = 120  # Export indices 100-119 (20 sequences)
output_path_range = export_dir / "sequence_range.jsonl"

dataset_manager.export.export_sequence_range(
    start_idx=start_idx,
    end_idx=end_idx,
    output_path=str(output_path_range),
    format_type="jsonl",
    return_detokenized=True,
    tokenizer=tokenizer,
    include_doc_details=True
)

print(f"Exported sequence range [{start_idx}:{end_idx}] to: {output_path_range}")

# Verify the range export
with open(output_path_range, 'r') as f:
    lines = f.readlines()
    print(f"Exported {len(lines)} sequences")
    
    # Check first and last records
    first_record = json.loads(lines[0])
    last_record = json.loads(lines[-1])
    
    print(f"First record index: {first_record['index']}")
    print(f"Last record index: {last_record['index']}")
    print(f"First record content: {first_record['content'][:70]}...")
    print(f"First record doc_details: {first_record.get('doc_details', 'N/A')}")


=== Export Sequence Range Example ===


TypeError: Object of type int32 is not JSON serializable

In [ ]:
# Example 6: Export batch range
print("\n=== Export Batch Range Example ===")

# Export a range of consecutive batches
start_batch = 3
end_batch = 8  # Export batch IDs 3, 4, 5, 6, 7 (5 batches)
batch_size = 2
output_path_batch_range = export_dir / "batch_range.csv"

dataset_manager.export.export_batch_range(
    start_batch=start_batch,
    end_batch=end_batch,
    batch_size=batch_size,
    output_path=str(output_path_batch_range),
    format_type="csv",
    return_detokenized=True,
    tokenizer=tokenizer,
    include_doc_details=False,
    flatten_batches=True  # Flatten for easier CSV reading
)

print(f"Exported batch range [{start_batch}:{end_batch}] with batch_size={batch_size} to: {output_path_batch_range}")

# Read and analyze the CSV
df_batch_range = pd.read_csv(output_path_batch_range)
print(f"Exported {len(df_batch_range)} total samples")
print(f"Expected samples: {(end_batch - start_batch) * batch_size} ({end_batch - start_batch} batches × {batch_size} samples each)")
print(f"Index range: {df_batch_range['index'].min()} to {df_batch_range['index'].max()}")
print(f"Sample from batch range:")
print(f"  Index {df_batch_range.iloc[0]['index']}: {df_batch_range.iloc[0]['content'][:60]}...")


=== Export Batch Range Example ===
Exported batch range [3:8] with batch_size=2 to: /NS/llm-pretraining/work/afkhan/tokensmith/exports/batch_range.csv
Exported 10 total samples
Expected samples: 10 (5 batches × 2 samples each)
Index range: 0 to 9
Sample from batch range:
  Index 0:  baby, saving them from certain injury. The creative baby fe...


In [ ]:
# Example 7: Export dataset range with chunking
print("\n=== Export Dataset Range with Chunking Example ===")

# Export a large range of the dataset using memory-efficient chunking
start_idx = 200
end_idx = 250  # Export 50 sequences
output_path_dataset_range = export_dir / "dataset_range_chunked.jsonl"

dataset_manager.export.export_dataset_range(
    start_idx=start_idx,
    end_idx=end_idx,
    output_path=str(output_path_dataset_range),
    format_type="jsonl",
    return_detokenized=True,
    tokenizer=tokenizer,
    include_doc_details=True,
    chunk_size=10  # Process in chunks of 10 for demonstration
)

print(f"Exported dataset range [{start_idx}:{end_idx}] with chunking to: {output_path_dataset_range}")

# Verify chunked export
with open(output_path_dataset_range, 'r') as f:
    lines = f.readlines()
    print(f"Total records exported: {len(lines)}")
    print(f"Expected records: {end_idx - start_idx}")
    
    # Show every 10th record to demonstrate chunking worked
    for i in range(0, min(len(lines), 30), 10):
        record = json.loads(lines[i])
        print(f"Record at position {i}: Index {record['index']}, Content: {record['content'][:50]}...")


=== Export Dataset Range with Chunking Example ===


TypeError: object of type 'WriteableMMapIndexedDataset' has no len()

In [ ]:
# Example 8: Export raw tokens (non-detokenized)
print("\n=== Export Raw Tokens Example ===")

# Export sequences as raw token arrays instead of detokenized text
sequence_indices_tokens = [20, 21, 22]
output_path_tokens = export_dir / "raw_tokens.jsonl"

dataset_manager.export.export_sequences(
    sequence_indices=sequence_indices_tokens,
    output_path=str(output_path_tokens),
    format_type="jsonl",
    return_detokenized=False,  # Export raw tokens
    tokenizer=None,  # Not needed when return_detokenized=False
    include_doc_details=False
)

print(f"Exported {len(sequence_indices_tokens)} sequences as raw tokens to: {output_path_tokens}")

# Read and display token arrays
with open(output_path_tokens, 'r') as f:
    for i, line in enumerate(f):
        record = json.loads(line)
        tokens = record['content']
        print(f"Sequence {record['index']}: {len(tokens)} tokens")
        print(f"  First 20 tokens: {tokens[:20]}")
        print(f"  Last 10 tokens: {tokens[-10:]}")
        print(f"  Token range: {min(tokens)} to {max(tokens)}")
        print()


=== Export Raw Tokens Example ===


AttributeError: 'list' object has no attribute 'tolist'

In [ ]:
# Example 9: Compare JSONL vs CSV formats
print("\n=== Compare Export Formats Example ===")

# Export the same data in both formats for comparison
comparison_indices = [80, 85, 90]

# Export as JSONL
jsonl_path = export_dir / "format_comparison.jsonl"
dataset_manager.export.export_sequences(
    sequence_indices=comparison_indices,
    output_path=str(jsonl_path),
    format_type="jsonl",
    return_detokenized=True,
    tokenizer=tokenizer,
    include_doc_details=True
)

# Export as CSV
csv_path = export_dir / "format_comparison.csv"
dataset_manager.export.export_sequences(
    sequence_indices=comparison_indices,
    output_path=str(csv_path),
    format_type="csv",
    return_detokenized=True,
    tokenizer=tokenizer,
    include_doc_details=True
)

print(f"Exported to both formats:")
print(f"  JSONL: {jsonl_path}")
print(f"  CSV: {csv_path}")

# Compare file sizes
import os
jsonl_size = os.path.getsize(jsonl_path)
csv_size = os.path.getsize(csv_path)
print(f"\nFile sizes:")
print(f"  JSONL: {jsonl_size} bytes")
print(f"  CSV: {csv_size} bytes")
print(f"  Size ratio (CSV/JSONL): {csv_size/jsonl_size:.2f}")

# Show structure differences
print(f"\nJSONL structure (first record):")
with open(jsonl_path, 'r') as f:
    first_jsonl = json.loads(f.readline())
    print(f"  Keys: {list(first_jsonl.keys())}")
    print(f"  Index: {first_jsonl['index']}")
    print(f"  Content length: {len(first_jsonl['content'])}")

print(f"\nCSV structure:")
df_comp = pd.read_csv(csv_path)
print(f"  Columns: {list(df_comp.columns)}")
print(f"  Shape: {df_comp.shape}")
print(f"  First row index: {df_comp.iloc[0]['index']}")
print(f"  First row content length: {len(df_comp.iloc[0]['content'])}")

In [ ]:
# Example 10: Export summary and file management
print("\n=== Export Summary and File Management ===")

# List all export files created
export_files = list(export_dir.glob("*"))
print(f"Created {len(export_files)} export files:")

total_size = 0
for file_path in sorted(export_files):
    if file_path.is_file():
        size = file_path.stat().st_size
        total_size += size
        print(f"  {file_path.name}: {size} bytes ({size/1024:.1f} KB)")

print(f"\nTotal export size: {total_size} bytes ({total_size/1024:.1f} KB)")

# Show file formats summary
jsonl_files = list(export_dir.glob("*.jsonl"))
csv_files = list(export_dir.glob("*.csv"))

print(f"\nFile format summary:")
print(f"  JSONL files: {len(jsonl_files)}")
print(f"  CSV files: {len(csv_files)}")

# Optional: Clean up demo files (uncomment to remove files)
# print(f"\nCleaning up demo export files...")
# for file_path in export_files:
#     if file_path.is_file():
#         file_path.unlink()
#         print(f"  Removed: {file_path.name}")

print(f"\nExport examples completed successfully!")
print(f"Check the '{export_dir}' directory to see all exported files.")